In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime

In [32]:
#an existing data set
df = pd.read_csv('Jurmala20210313.csv')

In [33]:
len(df)

269

In [34]:
existing = []
for i in df.Url:
    existing.append(i[17:])

In [35]:
#Loop through pages
urls = []
urls.append('https://www.ss.lv/lv/real-estate/flats/jurmala/sell/')
for i in range(1, 16):
    dz = "https://www.ss.lv/lv/real-estate/flats/jurmala/sell/page"+str(i)+".html"
    urls.append(dz)    

In [37]:
#Include all Jurmala. After EDA can reduce it to +- 5 km.m 
a = []
for url in urls:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    for link in soup.find_all('a', href=True):
        if link['href'].startswith( '/msg' ) and link['href'] not in a and link['href'] not in existing:
            a.append(link['href'])

In [39]:
url_ss = "https://www.ss.lv"
for i in a:
    ex = []
    bildes = []
    full_web_address = url_ss+i
    #print(full_web_address)
    ex.append(full_web_address)
    r = requests.get(full_web_address)
    data = r.text
    soup = BeautifulSoup(data)
    #print()    
    table_MN = pd.read_html(full_web_address)

    frames = [table_MN[3], table_MN[4], table_MN[5]]
    result = pd.concat(frames)

    for i in ['Pilsēta:', 'Rajons:', 'Iela:', 'Istabas:','Platība:','Stāvs:', 'Sērija:', 'Mājas tips:', 'Ērtības:','Cena:']:
        if i in result[0].unique():
            ex.append(result.loc[result[0] == i, (1)].item())
        else:
            ex.append('FALSE')
    if table_MN[7].shape[0] == 4:
        ex.append(table_MN[7][2][1][8:])
        ex.append(table_MN[7][2][2][28:])
    else:
        ex.append(table_MN[8][2][1][8:])
        ex.append(table_MN[8][2][2][28:])
    #Datums
    ex.append(datetime.datetime.now())
    #Advertisment text. Just in case. Todays mission is just to get it working. run now analyse later. 
    ex.append(str(soup.find_all("div", {"id": "msg_div_msg"})))
    
    for link in soup.find_all('a', href=True):
        if 'gallery' in link['href']:
            bildes.append(link['href'])
    ex.append(bildes)       
    df.loc[len(df)]= ex

In [17]:
df.to_csv('Jurmala20210313.csv',index = False)

In [31]:
for column in df.columns[1:-4]:
    print(column)
    print(df[column].value_counts())
    print()

Pilsēta
Jūrmala    269
Name: Pilsēta, dtype: int64

Rajons
Bulduri        62
Dzintari       55
Kauguri        40
Majori         25
Lielupe        19
Dubulti        17
Pumpuri        15
Melluži        11
Asari          10
Jaundubulti     9
Sloka           4
Buļļuciems      1
Ķemeri          1
Name: Rajons, dtype: int64

Iela
Dzintaru prospekts 48 [Karte]     6
Jasminu 9 [Karte]                 6
Asaru prospekts 57 [Karte]        5
Rigas iela 90 [Karte]             4
Mežaparka prospekts 10 [Karte]    4
                                 ..
Tīklu iela 11 [Karte]             1
Slokas iela 63/8 [Karte]          1
Turaidas iela 17 [Karte]          1
Bulduru prospekts 17 [Karte]      1
Hercoga Jekaba 10 [Karte]         1
Name: Iela, Length: 193, dtype: int64

Istabas
2    91
3    86
4    45
3    15
1    14
2     9
4     5
1     1
5     1
6     1
5     1
Name: Istabas, dtype: int64

Platība
50 m²        10
47 m²         7
55 m²         7
52 m²         7
64 m²         6
             ..
102.10 m² 

# THIS IS GOLD

In [ ]:
datums = pd.to_datetime('2021-02-15 13:39:51')
datumi = []
for i in range(0,len(df)):
    x = datums + datetime.timedelta(seconds=i)
    print(x)
    datumi.append(str(x))

In [ ]:
import numpy as np

df['web_date'] = np.resize(datumi,len(df))

In [ ]:
df.web_date.describe()

In [ ]:
https://www.ss.lv//msg/lv/real-estate/flats/jurmala/dzintari/agdic.html#photo-11